# carpark_data_gov_sg_api

## Purpose:
Explore the carpark availaability APIs at https://developers.data.gov.sg.

## History:
- 2018-04-18 - Shi Yuan: API Key not required since Feb 2018, code changes

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import requests
import seaborn as sns
from tqdm import tqdm

%matplotlib inline

In [2]:
# Not required since Feb 2018

## Get my API keys
#from my_api_keys import my_api_dict
## Note: this module, containing my API keys, will not be shared via GitHub
## You can obtain your own API key(s) by registering at https://developers.data.gov.sg
#my_key = my_api_dict['data.gov.sg']  # API key for data.gov.sg

## Meta-data for available meteorological APIs
[I added this section after exploring the wind-speed data - see below.]

In [3]:
# Meteorological variables
for variable in tqdm(['rainfall', 'wind-speed', 'wind-direction', 'air-temperature', 'relative-humidity']):
    print(variable)
    r = requests.get('https://api.data.gov.sg/v1/environment/{}'.format(variable))
    metadata = r.json()['metadata']
    for key in metadata.keys():
        if key != 'stations':  # don't print information about stations
            print('  {}: {}'.format(key, r.json()['metadata'][key]))

  0%|          | 0/5 [00:00<?, ?it/s]

rainfall
  reading_type: TB1 Rainfall 5 Minute Total F
  reading_unit: mm


 20%|██        | 1/5 [00:00<00:00,  4.95it/s]

wind-speed
  reading_type: Wind Speed AVG(S)10M M1M
  reading_unit: knots


 40%|████      | 2/5 [00:00<00:00,  4.26it/s]

...

 60%|██████    | 3/5 [00:00<00:00,  5.18it/s]

air-temperature
  reading_type: DBT 1M F
  reading_unit: deg C


 80%|████████  | 4/5 [00:00<00:00,  5.09it/s]

relative-humidity
  reading_type: RH 1M F
  reading_unit: percentage


100%|██████████| 5/5 [00:01<00:00,  4.70it/s]


In [4]:
# Meteorological variables
for variable in ['rainfall', 'wind-speed', 'wind-direction', 'air-temperature', 'relative-humidity']:
    print(variable)
    r = requests.get('https://api.data.gov.sg/v1/environment/{}'.format(variable))
    metadata = r.json()['metadata']
    for key in metadata.keys():
        if key == 'stations':  # print information about stations
            print('  {}: {}'.format(key, r.json()['metadata'][key]))

rainfall
  stations: [{'id': 'S77', 'device_id': 'S77', 'name': 'Alexandra Road', 'location': {'latitude': 1.2937, 'longitude': 103.8125}}, {'id': 'S109', 'device_id': 'S109', 'name': 'Ang Mo Kio Avenue 5', 'location': {'latitude': 1.3764, 'longitude': 103.8492}}, {'id': 'S117', 'device_id': 'S117', 'name': 'Banyan Road', 'location': {'latitude': 1.256, 'longitude': 103.679}}, {'id': 'S55', 'device_id': 'S55', 'name': 'Buangkok Green', 'location': {'latitude': 1.3837, 'longitude': 103.886}}, {'id': 'S64', 'device_id': 'S64', 'name': 'Bukit Panjang Road', 'location': {'latitude': 1.3824, 'longitude': 103.7603}}, {'id': 'S90', 'device_id': 'S90', 'name': 'Bukit Timah Road', 'location': {'latitude': 1.3191, 'longitude': 103.8191}}, {'id': 'S61', 'device_id': 'S61', 'name': 'Chai Chee Street', 'location': {'latitude': 1.323, 'longitude': 103.9217}}, {'id': 'S114', 'device_id': 'S114', 'name': 'Choa Chu Kang Avenue 4', 'location': {'latitude': 1.38, 'longitude': 103.73}}, {'id': 'S11', 'dev

  stations: [{'id': 'S109', 'device_id': 'S109', 'name': 'Ang Mo Kio Avenue 5', 'location': {'latitude': 1.3764, 'longitude': 103.8492}}, {'id': 'S117', 'device_id': 'S117', 'name': 'Banyan Road', 'location': {'latitude': 1.256, 'longitude': 103.679}}, {'id': 'S50', 'device_id': 'S50', 'name': 'Clementi Road', 'location': {'latitude': 1.3337, 'longitude': 103.7768}}, {'id': 'S107', 'device_id': 'S107', 'name': 'East Coast Parkway', 'location': {'latitude': 1.3135, 'longitude': 103.9625}}, {'id': 'S43', 'device_id': 'S43', 'name': 'Kim Chuan Road', 'location': {'latitude': 1.3399, 'longitude': 103.8878}}, {'id': 'S44', 'device_id': 'S44', 'name': 'Nanyang Avenue', 'location': {'latitude': 1.34583, 'longitude': 103.68166}}, {'id': 'S106', 'device_id': 'S106', 'name': 'Pulau Ubin', 'location': {'latitude': 1.4168, 'longitude': 103.9673}}, {'id': 'S102', 'device_id': 'S102', 'name': 'Semakau Landfill', 'location': {'latitude': 1.189, 'longitude': 103.768}}, {'id': 'S122', 'device_id': 'S12

In [260]:
#metadata.keys()

## Specify Date

In [261]:
date = '2018-01-08'
datetime = date + ' 00:05:00'
datetime

'2018-01-08 00:05:00'

## Carpark Availability - debug

In [130]:
# Get carpark availability at 30min interval - debug periods
carpark_df = pd.DataFrame(columns=['lot_type', 'total_lots', 'lots_available', 'timestamp (SGT)'])
for dt in tqdm(pd.date_range(datetime, periods=1, freq='30min')):  # I remember this was a wet day
    r = requests.get('https://api.data.gov.sg/v1/transport/carpark-availability',
                     params={'date_time': dt.strftime('%Y-%m-%dT%H:%M:%S')})
    temp_df = pd.DataFrame(r.json()['items'][0]['carpark_data'])
    #temp_df = temp_df.rename(columns={'lots_available': 'lots_available'})
    temp_df['timestamp (SGT)'] = pd.to_datetime(r.json()['items'][0]['timestamp'].split('+')[0])
    carpark_df = carpark_df.append(temp_df, ignore_index=True)
carpark_df.head()

100%|██████████| 1/1 [00:01<00:00,  1.56s/it]


,carpark_info,carpark_number,lot_type,lots_available,timestamp (SGT),total_lots,update_datetime
0,"[{'total_lots': '104', 'lot_type': 'C', 'lots_...",HE12,NaN,NaN,2018-04-17 00:04:28,NaN,2018-04-17T00:03:23
1,"[{'total_lots': '583', 'lot_type': 'C', 'lots_...",HLM,NaN,NaN,2018-04-17 00:04:28,NaN,2018-04-17T00:03:40
2,"[{'total_lots': '329', 'lot_type': 'C', 'lots_...",RHM,NaN,NaN,2018-04-17 00:04:28,NaN,2018-04-17T00:03:21
3,"[{'total_lots': '110', 'lot_type': 'C', 'lots_...",BM29,NaN,NaN,2018-04-17 00:04:28,NaN,2017-08-01T12:15:17
4,"[{'total_lots': '97', 'lot_type': 'C', 'lots_a...",Q81,NaN,NaN,2018-04-17 00:04:28,NaN,2018-04-17T00:03:38


## Read carpark info file

In [224]:
info_df = pd.read_csv("hdb-carpark-information.csv")
info_df.head()

,car_park_no,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO
4,AK31,BLK 302/348 ANG MO KIO ST 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO


In [225]:
info_df = info_df.rename(columns={'car_park_no': 'carpark_number'})
#info_df = info_df.rename(columns={'x_coord': 'longitude'})
#info_df = info_df.rename(columns={'y_coord': 'latitude'})
info_df

,carpark_number,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO
4,AK31,BLK 302/348 ANG MO KIO ST 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO
5,AK52,BLOCK 513 ANG MO KIO ST 53,29889.3457,39382.8134,SURFACE CAR PARK,COUPON PARKING,WHOLE DAY,NO,YES
6,AK6,BLOCK 728 ANG MO KIO AVE 6,29283.7324,39647.8906,SURFACE CAR PARK,COUPON PARKING,7AM-10.30PM,SUN & PH FR 7AM-10.30PM,NO
7,AK83,BLK 5022 TO 5095 ANG MO KIO INDUSTRIAL PARK 2,31397.2241,39851.6191,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO
8,AK9,ANG MO KIO AVENUE 9,29674.8184,40616.8750,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO
9,AM16,BLK 256A ANG MO KIO STREET 21,28267.0582,39151.8344,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES


In [232]:
def SVY21toWGS84(X,Y):
    r = requests.get('https://developers.onemap.sg/commonapi/convert/3414to4326',
                     params={'X': X, 'Y':Y})
    return r.json()

In [236]:
info_df['x_coord'][0]

30314.7936

In [237]:
info_df['y_coord'][0]

31490.4942

In [249]:
result = SVY21toWGS84(info_df['x_coord'][1],info_df['y_coord'][1])
result['longitude']
result['latitude']

1.321004290105958

In [235]:
len(info_df)

2066

In [254]:
info_df['longitude']=''
info_df['latitude']=''
for i in tqdm(range(0,len(info_df))):
    result = SVY21toWGS84(info_df['x_coord'][i],info_df['y_coord'][i])
    info_df['longitude'][i]= result['longitude']
    info_df['latitude'][i]= result['latitude']
info_df

  0%|          | 0/2066 [00:00<?, ?it/s]C:\Users\User\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\User\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
100%|██████████| 2066/2066 [05:17<00:00,  6.51it/s]


In [255]:
info_df

,carpark_number,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,longitude,latitude
0,ACB,BLK 270/271 ALBERT CENTRE BASEMENT CAR PARK,30314.7936,31490.4942,BASEMENT CAR PARK,ELECTRONIC PARKING,WHOLE DAY,NO,YES,103.854,1.30106
1,ACM,BLK 98A ALJUNIED CRESCENT,33758.4143,33695.5198,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,103.885,1.321
2,AH1,BLK 101 JALAN DUSUN,29257.7203,34500.3599,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,103.845,1.32828
3,AK19,BLOCK 253 ANG MO KIO STREET 21,28185.4359,39012.6664,SURFACE CAR PARK,COUPON PARKING,7AM-7PM,NO,NO,103.835,1.36909
4,AK31,BLK 302/348 ANG MO KIO ST 31,29482.0290,38684.1754,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,103.847,1.36612
5,AK52,BLOCK 513 ANG MO KIO ST 53,29889.3457,39382.8134,SURFACE CAR PARK,COUPON PARKING,WHOLE DAY,NO,YES,103.85,1.37244
6,AK6,BLOCK 728 ANG MO KIO AVE 6,29283.7324,39647.8906,SURFACE CAR PARK,COUPON PARKING,7AM-10.30PM,SUN & PH FR 7AM-10.30PM,NO,103.845,1.37484
7,AK83,BLK 5022 TO 5095 ANG MO KIO INDUSTRIAL PARK 2,31397.2241,39851.6191,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,103.864,1.37668
8,AK9,ANG MO KIO AVENUE 9,29674.8184,40616.8750,SURFACE CAR PARK,COUPON PARKING,NO,NO,NO,103.848,1.3836
9,AM16,BLK 256A ANG MO KIO STREET 21,28267.0582,39151.8344,MULTI-STOREY CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,103.836,1.37035


In [227]:
r.json()['latitude']

1.3197295716669164

ValueError: DataFrame constructor not properly called!

In [108]:
r.json()

{'items': [{'carpark_data': [{'carpark_info': [{'lot_type': 'C',
       'lots_available': '62',
       'total_lots': '104'}],
     'carpark_number': 'HE12',
     'update_datetime': '2018-04-17T00:03:23'},
    {'carpark_info': [{'lot_type': 'C',
       'lots_available': '465',
       'total_lots': '583'}],
     'carpark_number': 'HLM',
     'update_datetime': '2018-04-17T00:03:40'},
    {'carpark_info': [{'lot_type': 'C',
       'lots_available': '113',
       'total_lots': '329'}],
     'carpark_number': 'RHM',
     'update_datetime': '2018-04-17T00:03:21'},
    {'carpark_info': [{'lot_type': 'C',
       'lots_available': '109',
       'total_lots': '110'}],
     'carpark_number': 'BM29',
     'update_datetime': '2017-08-01T12:15:17'},
    {'carpark_info': [{'lot_type': 'C',
       'lots_available': '71',
       'total_lots': '97'}],
     'carpark_number': 'Q81',
     'update_datetime': '2018-04-17T00:03:38'},
    {'carpark_info': [{'lot_type': 'C',
       'lots_available': '150',
    

In [114]:
#metadata = r.json()['metadata']

In [7]:
pd.date_range(date, periods=2, freq='30min')

DatetimeIndex(['2018-04-17 00:05:00', '2018-04-17 00:35:00'], dtype='datetime64[ns]', freq='30T')

## Carpark Availability - 30min interval

In [262]:
# Get carpark availability at 30-min intervals on a specific date
carpark_df = pd.DataFrame(columns=['lot_type', 'total_lots', 'lots_available', 'timestamp (SGT)'])
for dt in tqdm(pd.date_range(datetime,  periods=(24*(60/30)+1), freq='30min')):  # I remember this was a wet day
    r = requests.get('https://api.data.gov.sg/v1/transport/carpark-availability',
                     params={'date_time': dt.strftime('%Y-%m-%dT%H:%M:%S')})
    try:
        temp_df = pd.DataFrame(r.json()['items'][0]['carpark_data'])
    except ValueError:
        print('No data JSONDecodeError {}'.format(dt))
        continue
    except KeyError:
        print('No items KeyError {}'.format(dt))
        continue
    #temp_df = temp_df.rename(columns={'lots_available': 'lots_available'})
    temp_df['timestamp (SGT)'] = pd.to_datetime(r.json()['items'][0]['timestamp'].split('+')[0])
    carpark_df = carpark_df.append(temp_df, ignore_index=True)
carpark_df.head()

 84%|████████▎ | 41/49 [00:43<00:08,  1.07s/it]

No items KeyError 2018-01-08 20:35:00


 94%|█████████▍| 46/49 [01:35<00:06,  2.07s/it]

No items KeyError 2018-01-08 23:05:00


100%|██████████| 49/49 [02:00<00:00,  2.47s/it]


,carpark_info,carpark_number,lot_type,lots_available,timestamp (SGT),total_lots,update_datetime
0,"[{'total_lots': '104', 'lot_type': 'C', 'lots_...",HE12,NaN,NaN,2018-01-08 00:04:27,NaN,2018-01-08T00:03:19
1,"[{'total_lots': '583', 'lot_type': 'C', 'lots_...",HLM,NaN,NaN,2018-01-08 00:04:27,NaN,2018-01-08T00:03:12
2,"[{'total_lots': '329', 'lot_type': 'C', 'lots_...",RHM,NaN,NaN,2018-01-08 00:04:27,NaN,2018-01-08T00:03:07
3,"[{'total_lots': '110', 'lot_type': 'C', 'lots_...",BM29,NaN,NaN,2018-01-08 00:04:27,NaN,2017-08-01T12:15:17
4,"[{'total_lots': '97', 'lot_type': 'C', 'lots_a...",Q81,NaN,NaN,2018-01-08 00:04:27,NaN,2018-01-08T00:03:27


In [133]:
pd.date_range(date,  periods=(24*(60/30)+1), freq='30min')

DatetimeIndex(['2018-04-17 00:00:00', '2018-04-17 00:30:00',
               '2018-04-17 01:00:00', '2018-04-17 01:30:00',
               '2018-04-17 02:00:00', '2018-04-17 02:30:00',
               '2018-04-17 03:00:00', '2018-04-17 03:30:00',
               '2018-04-17 04:00:00', '2018-04-17 04:30:00',
               '2018-04-17 05:00:00', '2018-04-17 05:30:00',
               '2018-04-17 06:00:00', '2018-04-17 06:30:00',
               '2018-04-17 07:00:00', '2018-04-17 07:30:00',
               '2018-04-17 08:00:00', '2018-04-17 08:30:00',
               '2018-04-17 09:00:00', '2018-04-17 09:30:00',
               '2018-04-17 10:00:00', '2018-04-17 10:30:00',
               '2018-04-17 11:00:00', '2018-04-17 11:30:00',
               '2018-04-17 12:00:00', '2018-04-17 12:30:00',
               '2018-04-17 13:00:00', '2018-04-17 13:30:00',
               '2018-04-17 14:00:00', '2018-04-17 14:30:00',
               '2018-04-17 15:00:00', '2018-04-17 15:30:00',
               '2018-04-

## Carpark Availability - 5min interval

In [211]:
# Get carpark availability at 5-min intervals on a specific date
carpark_df = pd.DataFrame(columns=['lot_type', 'total_lots', 'lots_available', 'timestamp (SGT)'])
for dt in pd.date_range(datetime,  periods=(24*(60/5)+1), freq='5min'):  # I remember this was a wet day
    r = requests.get('https://api.data.gov.sg/v1/transport/carpark-availability',
                     params={'date_time': dt.strftime('%Y-%m-%dT%H:%M:%S')})
    try:
        temp_df = pd.DataFrame(r.json()['items'][0]['carpark_data'])
        #temp_df = temp_df.rename(columns={'lots_available': 'lots_available'})
    except ValueError:
        print('No data JSONDecodeError {}'.format(dt))
        continue
    except KeyError:
        print('No items KeyError {}'.format(dt))
        continue
    temp_df['timestamp (SGT)'] = pd.to_datetime(r.json()['items'][0]['timestamp'].split('+')[0])
    carpark_df = carpark_df.append(temp_df, ignore_index=True)
carpark_df.head()

IndexError: list index out of range

In [263]:
carpark_df.head()

,carpark_info,carpark_number,lot_type,lots_available,timestamp (SGT),total_lots,update_datetime
0,"[{'total_lots': '104', 'lot_type': 'C', 'lots_...",HE12,NaN,NaN,2018-01-08 00:04:27,NaN,2018-01-08T00:03:19
1,"[{'total_lots': '583', 'lot_type': 'C', 'lots_...",HLM,NaN,NaN,2018-01-08 00:04:27,NaN,2018-01-08T00:03:12
2,"[{'total_lots': '329', 'lot_type': 'C', 'lots_...",RHM,NaN,NaN,2018-01-08 00:04:27,NaN,2018-01-08T00:03:07
3,"[{'total_lots': '110', 'lot_type': 'C', 'lots_...",BM29,NaN,NaN,2018-01-08 00:04:27,NaN,2017-08-01T12:15:17
4,"[{'total_lots': '97', 'lot_type': 'C', 'lots_a...",Q81,NaN,NaN,2018-01-08 00:04:27,NaN,2018-01-08T00:03:27


In [10]:
#dt.strftime('%Y-%m-%dT%H:%M:%S')

In [11]:
#r.json()

In [12]:
#r.json()['items'][0]['carpark_data']

In [13]:
#temp_df = pd.DataFrame(r.json()['items'][0]['carpark_data'])


In [264]:
carpark_df['lots_available']=''

In [15]:
#type(carpark_df['carpark_info'][0])

In [16]:
#type(carpark_df['carpark_info'][0][0])

In [17]:
#carpark_df['carpark_info'][1][0]

In [136]:
len(carpark_df)

73230

In [137]:
#carpark_df['lot_type']

In [265]:
carpark_df['carpark_info'][3][0]['lot_type']

'C'

In [266]:
carpark_df['carpark_info'].head()

0    [{'total_lots': '104', 'lot_type': 'C', 'lots_...
1    [{'total_lots': '583', 'lot_type': 'C', 'lots_...
2    [{'total_lots': '329', 'lot_type': 'C', 'lots_...
3    [{'total_lots': '110', 'lot_type': 'C', 'lots_...
4    [{'total_lots': '97', 'lot_type': 'C', 'lots_a...
Name: carpark_info, dtype: object

In [268]:
type(carpark_df['carpark_info'][0][0])

dict

In [269]:
df3 = pd.DataFrame(carpark_df['carpark_info'].values.tolist(), columns=['carpark_info1','carpark_info2','carpark_info3'])
df3.head()

,carpark_info1,carpark_info2,carpark_info3
0,"{'total_lots': '104', 'lot_type': 'C', 'lots_a...",None,None
1,"{'total_lots': '583', 'lot_type': 'C', 'lots_a...",None,None
2,"{'total_lots': '329', 'lot_type': 'C', 'lots_a...",None,None
3,"{'total_lots': '110', 'lot_type': 'C', 'lots_a...",None,None
4,"{'total_lots': '97', 'lot_type': 'C', 'lots_av...",None,None


In [270]:
df3['carpark_info1'].head()

0    {'total_lots': '104', 'lot_type': 'C', 'lots_a...
1    {'total_lots': '583', 'lot_type': 'C', 'lots_a...
2    {'total_lots': '329', 'lot_type': 'C', 'lots_a...
3    {'total_lots': '110', 'lot_type': 'C', 'lots_a...
4    {'total_lots': '97', 'lot_type': 'C', 'lots_av...
Name: carpark_info1, dtype: object

In [271]:
type(df3['carpark_info1'][0])

dict

In [272]:
json.dumps(df3['carpark_info1'][0])

'{"total_lots": "104", "lot_type": "C", "lots_available": "39"}'

In [273]:
df3['carpark_info1'] = df3['carpark_info1'].apply(json.dumps)

In [274]:
type(df3['carpark_info1'][0])

str

In [164]:
#str(df3['carpark_info1'][0])

In [165]:
#df3['carpark_info1'].astype(str)

In [275]:
temp_df1, temp_df2, temp_df3 = df3['carpark_info1'].str.split(',').str

In [193]:
#temp_df3

In [194]:
#df3['carpark_info1'].head()
#temp_df1, temp_df2, temp_df3 = df3['carpark_info1'].astype(str).str.split(',').str

In [276]:
temp_df1.head()

0    {"total_lots": "104"
1    {"total_lots": "583"
2    {"total_lots": "329"
3    {"total_lots": "110"
4     {"total_lots": "97"
Name: carpark_info1, dtype: object

In [68]:
#str = "h3110 23 cat 444.4 rabbit 11 2 dog"
#[int(s) for s in str.split() if s.isdigit()]

In [277]:
temp_df1[0]

'{"total_lots": "104"'

In [278]:
str = temp_df1[0]
[int(s) for s in str.split('\"') if s.isdigit()]

[104]

## Define function to extract number from string

In [279]:
def extract_number(x):
    str = x
    return [int(s) for s in str.split('\"') if s.isdigit()][0]

In [280]:
extract_number(temp_df1[0])

104

In [281]:
# total lots
temp_df1.apply(extract_number).head()

0    104
1    583
2    329
3    110
4     97
Name: carpark_info1, dtype: int64

In [282]:
str = temp_df2[0]
[s for s in str.split('\"') if s.isalpha()]

['C']

## Define function to extract single character from string

In [283]:
def extract_alpha(x):
    str = x
    return [s for s in str.split('\"') if s.isalpha()][0]

In [284]:
extract_alpha(temp_df2[0])

'C'

In [285]:
# lot type
temp_df2.apply(extract_alpha).head()
#temp_df2.head()

0    C
1    C
2    C
3    C
4    C
Name: carpark_info1, dtype: object

In [286]:
# lots available
temp_df3.apply(extract_number).head()

0     39
1    473
2    102
3    109
4     72
Name: carpark_info1, dtype: int64

In [287]:
#df3['carpark_info1'].head().str.split(',')

In [288]:
#temp_df1, temp_df2, temp_df3 = carpark_df['carpark_info'].str.split(',').str

In [289]:
#temp_df

## Method below using 'for-loop', too slow

In [290]:
#for i in tqdm(range(0,len(carpark_df))):
#    carpark_df['lot_type'][i] = carpark_df['carpark_info'][i][0]['lot_type']
#    carpark_df['total_lots'][i] = carpark_df['carpark_info'][i][0]['total_lots']
#    carpark_df['lots_available'][i] = carpark_df['carpark_info'][i][0]['lots_available']

## Vectorisation Method

In [291]:
carpark_df['lot_type'] = temp_df2.apply(extract_alpha)
carpark_df['total_lots'] = temp_df1.apply(extract_number)
carpark_df['lots_available'] = temp_df3.apply(extract_number)
carpark_df.head()

,carpark_info,carpark_number,lot_type,lots_available,timestamp (SGT),total_lots,update_datetime
0,"[{'total_lots': '104', 'lot_type': 'C', 'lots_...",HE12,C,39,2018-01-08 00:04:27,104,2018-01-08T00:03:19
1,"[{'total_lots': '583', 'lot_type': 'C', 'lots_...",HLM,C,473,2018-01-08 00:04:27,583,2018-01-08T00:03:12
2,"[{'total_lots': '329', 'lot_type': 'C', 'lots_...",RHM,C,102,2018-01-08 00:04:27,329,2018-01-08T00:03:07
3,"[{'total_lots': '110', 'lot_type': 'C', 'lots_...",BM29,C,109,2018-01-08 00:04:27,110,2017-08-01T12:15:17
4,"[{'total_lots': '97', 'lot_type': 'C', 'lots_a...",Q81,C,72,2018-01-08 00:04:27,97,2018-01-08T00:03:27


In [256]:
outer_df = pd.merge(carpark_df, info_df, how='outer', on=['carpark_number'])
outer_df.head(15)

,carpark_info,carpark_number,lot_type,lots_available,timestamp (SGT),total_lots,update_datetime,address,x_coord,y_coord,car_park_type,type_of_parking_system,short_term_parking,free_parking,night_parking,longitude,latitude
0,"[{'total_lots': '104', 'lot_type': 'C', 'lots_...",HE12,C,62.0,2018-04-17 00:04:28,104.0,2018-04-17T00:03:23,BLK 78/81 REDHILL LANE,26367.5806,30069.2434,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,103.819,1.28821
1,"[{'total_lots': '104', 'lot_type': 'C', 'lots_...",HE12,C,59.0,2018-04-17 00:34:30,104.0,2018-04-17T00:33:23,BLK 78/81 REDHILL LANE,26367.5806,30069.2434,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,103.819,1.28821
2,"[{'total_lots': '104', 'lot_type': 'C', 'lots_...",HE12,C,59.0,2018-04-17 01:04:29,104.0,2018-04-17T01:03:23,BLK 78/81 REDHILL LANE,26367.5806,30069.2434,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,103.819,1.28821
3,"[{'total_lots': '104', 'lot_type': 'C', 'lots_...",HE12,C,59.0,2018-04-17 01:34:28,104.0,2018-04-17T01:33:23,BLK 78/81 REDHILL LANE,26367.5806,30069.2434,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,103.819,1.28821
4,"[{'total_lots': '104', 'lot_type': 'C', 'lots_...",HE12,C,58.0,2018-04-17 02:04:28,104.0,2018-04-17T02:03:23,BLK 78/81 REDHILL LANE,26367.5806,30069.2434,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,103.819,1.28821
5,"[{'total_lots': '104', 'lot_type': 'C', 'lots_...",HE12,C,57.0,2018-04-17 02:34:28,104.0,2018-04-17T02:33:23,BLK 78/81 REDHILL LANE,26367.5806,30069.2434,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,103.819,1.28821
6,"[{'total_lots': '104', 'lot_type': 'C', 'lots_...",HE12,C,57.0,2018-04-17 03:04:28,104.0,2018-04-17T03:03:32,BLK 78/81 REDHILL LANE,26367.5806,30069.2434,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,103.819,1.28821
7,"[{'total_lots': '104', 'lot_type': 'C', 'lots_...",HE12,C,59.0,2018-04-17 03:34:27,104.0,2018-04-17T03:33:32,BLK 78/81 REDHILL LANE,26367.5806,30069.2434,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,103.819,1.28821
8,"[{'total_lots': '104', 'lot_type': 'C', 'lots_...",HE12,C,74.0,2018-04-17 04:04:28,104.0,2018-04-17T04:03:32,BLK 78/81 REDHILL LANE,26367.5806,30069.2434,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,103.819,1.28821
9,"[{'total_lots': '104', 'lot_type': 'C', 'lots_...",HE12,C,74.0,2018-04-17 04:34:28,104.0,2018-04-17T04:33:32,BLK 78/81 REDHILL LANE,26367.5806,30069.2434,SURFACE CAR PARK,ELECTRONIC PARKING,WHOLE DAY,SUN & PH FR 7AM-10.30PM,YES,103.819,1.28821


In [223]:
#r = requests.get('https://developers.onemap.sg/commonapi/convert/3414to4326?X={}&Y={}'.format(variable))

In [292]:
r = requests.get('https://developers.onemap.sg/commonapi/convert/3414to4326?X=28983.788791079794&Y=33554.5098132845')
r.json()

{'latitude': 1.3197295716669164, 'longitude': 103.84215843333567}

In [222]:
r = requests.get('https://developers.onemap.sg/commonapi/convert/3414to4326',
                     params={'X': '28983.788791079794', 'Y': '33554.5098132845'})
r.json()

{'latitude': 1.3197295716669164, 'longitude': 103.84215843333567}

In [295]:
filename = 'carpark_{}.csv'.format(date)
filename

outer_df.to_csv(filename, index = False)

In [249]:
carpark_df['lot_type'][0]

'C'

In [ ]:
carpark_df.head()

In [293]:
outer_df.describe()

,lots_available,total_lots,x_coord,y_coord
count,73230.000000,73230.000000,71075.000000,71075.000000
mean,197.334467,400.487054,28775.066967,37777.370982
std,197.524002,323.881305,7956.053479,4802.618377
min,0.000000,2.000000,11539.089800,28123.411600
25%,51.000000,180.000000,22252.486000,34382.230500
50%,149.000000,337.000000,29482.029000,37234.277300
75%,286.000000,530.000000,35011.040100,40693.989700
max,1999.000000,4443.000000,45264.580600,48691.430800


In [294]:
len(temp_df)

1668

In [184]:
#temp_df[['total_lots','lot_type','lots_available']] = pd.DataFrame(temp_df.carpark_info.values.tolist(), index= temp_df.index)